# Grid Radar -> Plot Max-CAPPI

We are using [fast-barnes-py](https://github.com/MeteoSwiss/fast-barnes-py.git) to grid the radar data. Please cite if you use it in your research.

In [ ]:
import fsspec
import radarx as rx
import xradar as xd
import cmweather  # noqa
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from open_radar_data import DATASETS

In [ ]:
file = "s3://noaa-nexrad-level2/2022/03/30/KGWX/KGWX20220330_234639_V06"
local_file = fsspec.open_local(
    f"simplecache::s3://{file}",
    s3={"anon": True},
    filecache={"cache_storage": "."},
)
dtree = xd.io.open_nexradlevel2_datatree(local_file)
dtree = rx.combine_nexrad_sweeps(dtree)
dtree = dtree.xradar.georeference()

In [ ]:
dtree.groups

In [ ]:
def filter_radar(ds):
    ds = ds.where((ds.DBZH > -10) & (ds.DBZH < 75))
    return ds

In [ ]:
dtree = dtree.xradar.map_over_sweeps(filter_radar)

In [ ]:
# Create a figure and axis
fig, ax = plt.subplots(figsize=(7, 5))
dtree["sweep_2"]["DBZH"].plot.contourf(
    x="x",
    y="y",
    levels=range(-10, 75),
    cmap="NWSRef",
    ylim=(-200e3, 300e3),  # Adjust y-axis limits
    xlim=(-200e3, 300e3),  # Adjust x-axis limits
    ax=ax,  # Use the created axis
)
# Set the title
ax.set_title(
    f"{dtree.attrs['instrument_name']} {dtree['sweep_0']['time'].min().values}"
)
# Show the plot
plt.show()

In [ ]:
%%time
ds = dtree.radarx.to_grid(
    data_vars=["DBZH"],
    pseudo_cappi=True,
    x_lim=(-100000.0, 100000.0),
    y_lim=(-100000.0, 100000.0),
    z_lim=(0, 10000.0),
    x_step=1000,
    y_step=1000,
    z_step=250,
    x_smth=0.2,
    y_smth=0.2,
    z_smth=1,
)

In [ ]:
ds.radarx.plot_max_cappi("DBZH", cmap="ChaseSpectral", add_slogan=True)

In [ ]:
%%time
ds2 = dtree.radarx.to_grid(
    data_vars=["DBZH"],
    pseudo_cappi=False,
    x_lim=(-200000.0, 200000.0),
    y_lim=(-200000.0, 200000.0),
    z_lim=(0, 15000.0),
    x_step=1000,
    y_step=1000,
    z_step=250,
    x_smth=0.2,
    y_smth=0.2,
    z_smth=1,
)

In [ ]:
ds2.radarx.plot_max_cappi("DBZH", cmap="ChaseSpectral", add_slogan=True)

In [ ]:
#!/usr/bin/env python
# Copyright (c) 2024-2025, Radarx developers.
# Distributed under the MIT License. See LICENSE for more info.

"""
Tests for Radarx Max-Cappi
==========================
"""
import os
import pytest
import matplotlib.pyplot as plt
from radarx.vis.maxcappi import plot_maxcappi
from open_radar_data import DATASETS
import xradar as xd
import radarx as rx


# @pytest.fixture
def mock_dtree():
    """Fixture to create a mock radar DataTree."""
    file = DATASETS.fetch("swx_20120520_0641.nc")
    dtree = xd.io.open_cfradial1_datatree(file, sweep=[0, 1, 2, 3, 5, 7])
    return dtree


def test_plot_maxcappi_basic(mock_dtree):
    """Basic test for `plot_maxcappi` with visualization."""
    # Grid the radar data
    ds = mock_dtree.radarx.to_grid(
        data_vars=["corrected_reflectivity_horizontal"],
        pseudo_cappi=True,
        x_lim=(-50e3, 50e3),
        y_lim=(-50e3, 50e3),
        z_lim=(0, 5e3),
        x_step=2000,
        y_step=2000,
        z_step=1000,
    )

    # Define a temporary directory to save the plot
    path = os.get_cwd()
    print(path)
    savedir = f"{path}/test_plot.png"
    print(savedir)

    # Plot Max-CAPPI with additional annotations
    plot_maxcappi(
        ds=ds,
        data_var="corrected_reflectivity_horizontal",
        cmap="viridis",
        vmin=0,
        vmax=60,
        title="Test Max-CAPPI with Visualization",
        add_map=True,
        colorbar=True,
        show_figure=True,  # Launch the figure for visualization
        savedir=savedir,
    )

    # Assert the plot was saved
    assert os.path.exists(savedir), "The Max-CAPPI plot was not saved as expected."

    # Visualize the saved figure for real-time debugging
    img = plt.imread(savedir)
    plt.figure()
    plt.imshow(img)
    plt.title("Saved Max-CAPPI Visualization")
    plt.axis("off")
    plt.show()

    # Add further assertions (e.g., validate dataset properties)
    assert (
        "corrected_reflectivity_horizontal" in ds.data_vars
    ), "Variable missing in dataset."
    assert ds["corrected_reflectivity_horizontal"].shape == (
        6,
        51,
        51,
    ), "Unexpected data shape."

In [ ]:
test_plot_maxcappi_basic(mock_dtree)